# Document Clustering and Topic Modeling

*In* this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [53]:
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [54]:
import pandas as pd

In [55]:
# https://drive.google.com/open?id=192JMR7SIqoa14vrs7Z9BXO3iK89pimJL
# file = drive.CreateFile({'id':'192JMR7SIqoa14vrs7Z9BXO3iK89pimJL'}) # replace the id with id of file you want to access
# file.GetContentFile('data.tsv') 
# df = pd.read_csv('watch_reviews.tsv')


# Part 1: Load Data

In [56]:
!pip install --upgrade pip
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

Requirement already up-to-date: pip in /Users/chang/opt/anaconda3/lib/python3.7/site-packages (20.2.4)


[nltk_data] Downloading package punkt to /Users/chang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/chang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
# Load data into dataframe
df = pd.read_csv('watch_reviews.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [58]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [59]:
# Remove missing value
df.review_body.dropna(inplace=True)

In [60]:
# use the first 1000 data as our training data
data = df.loc[:1000, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [61]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("n't")
stopwords.append("br")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 183 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [62]:
from nltk.stem.snowball import SnowballStemmer
# REGULAR EXPRESSION
import re

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

***example:***

document1: "Arthur da Jason"

document 2: "Jason da da huang"

document1: tf-idf [1, 0.5, 0.5, 0];  document2: tf-idf [0, 1, 0.5, 1]  

2-gram: 

document 1: Arthur da, da Jason; document 2: Jason da, da da, da huang bigram

3-gram:

document 1: Athur da Jason;  document 2: Jason da da, da da huang

[Arhur, da, Jason...]

In [63]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/Users/chang/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 241 terms.


In [64]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming(text)>,
 'use_idf': True,
 'vocabulary': None}

Save the terms identified by TF-IDF.

In [65]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [66]:
# print out words
tf_selected_words

['abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi',
 'high',
 'hold',


# Part 4: K-means clustering

In [67]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [68]:
tfidf_matrix

<1000x241 sparse matrix of type '<class 'numpy.float64'>'
	with 7378 stored elements in Compressed Sparse Row format>

## 4.1. Analyze K-means Result

In [69]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].product_title, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [70]:
frame.head(10)

,review,cluster
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",2
1,Kenneth Cole New York Women's KC4944 Automatic...,2
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,0
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,0
4,Orient ER27009B Men's Symphony Automatic Stain...,0
5,Casio Men's GW-9400BJ-1JF G-Shock Master of G ...,2
6,Fossil Women's ES3851 Urban Traveler Multifunc...,1
7,INFANTRY Mens Night Vision Analog Quartz Wrist...,0
8,G-Shock Men's Grey Sport Watch,1
9,Heiden Quad Watch Winder in Black Leather,1


In [71]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
0,648
2,114
1,100
3,71
4,67


In [72]:
km.cluster_centers_

array([[0.00567646, 0.00444437, 0.00384839, ..., 0.00655119, 0.01757328,
        0.01326296],
       [0.00320692, 0.        , 0.        , ..., 0.0021164 , 0.00371946,
        0.02256662],
       [0.        , 0.03956174, 0.        , ..., 0.01185082, 0.01655174,
        0.00383501],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00918964,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00710388,
        0.        ]])

In [73]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

<Document clustering result by K-means>
Cluster 0 words:watch,like,look,band,work,time,
Cluster 0 reviews (648 reviews): 
Ritche 22mm Black Stainless Steel Bracelet Watch Band Strap Pebble Time/Pebble Classic, Citizen Men's BM8180-03E Eco-Drive Stainless Steel Watch with Green Canvas Band, Orient ER27009B Men's Symphony Automatic Stainless Steel Black Dial Mechanical Watch, INFANTRY Mens Night Vision Analog Quartz Wrist Watch with Nato Nylon Watchband-Red., Fossil Women's ES3621 Serena Crystal-Accented Two-Tone Stainless Steel Watch, Casio General Men's Watches Sporty Digital AE-2000W-1AVDF - WW, 2Tone Gold Silver Cable Band Ladies Bangle Cuff Watch, Bulova Men's 98B143 Precisionist Charcoal Grey Dial Bracelet Watch, Casio - G-Shock - Gulfmaster - Black - GWN1000C-1A, Invicta Men's 3329 Force Collection Lefty Watch, Seiko Women's SUT068 Dress Solar Classic Diamond-Accented Two-Tone Stainless Steel Watch, Anne Klein Women's 109271MPTT Swarovski Crystal Accented Two-Tone Multi-Chain Brac

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [74]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA requires integer values
tfidf_model_lda = CountVectorizer(max_df=0.99, max_features=500,
                                 min_df=0.01, stop_words='english',
                                 tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix_lda = tfidf_model_lda.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix_lda.shape[0]) + \
      " reviews and " + str(tfidf_matrix_lda.shape[1]) + " terms.")

/Users/chang/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 241 terms.


In [76]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix_lda)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.18752935 0.025925   0.7358677  0.02514762 0.02553033]
 [0.05114095 0.05247844 0.79437173 0.05000259 0.05200629]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000013 0.10069775 0.10000014 0.10000048 0.5993015 ]
 [0.79704166 0.05091076 0.05067541 0.0500036  0.05136856]
 [0.04011074 0.62870604 0.04124892 0.04020106 0.24973324]]


In [77]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 241)
[[ 9.9072546   0.20099944  6.66848972 ...  5.16159755 10.43774789
  10.51465353]
 [ 0.21057734  0.20091444  1.34381103 ...  0.24317919 10.4637319
  36.05301456]
 [ 0.20228558 12.65342947  0.20083052 ...  5.71581304 37.3957062
   0.20135795]
 [ 0.20206483  4.74260247  0.2011949  ...  2.95687826  0.20134865
   0.20044116]
 [ 6.47781765  0.20205418  4.58567383 ...  3.92253197 13.50146536
  10.0305328 ]]


In [78]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.19,0.03,0.74,0.03,0.03,2
Doc1,0.05,0.05,0.79,0.05,0.05,2
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.03,0.03,0.03,0.03,0.88,4
Doc4,0.01,0.51,0.07,0.01,0.41,1
Doc5,0.45,0.04,0.43,0.04,0.04,0
Doc6,0.42,0.03,0.49,0.03,0.03,2
Doc7,0.88,0.03,0.03,0.03,0.03,0
Doc8,0.39,0.50,0.01,0.08,0.01,1
Doc9,0.25,0.03,0.03,0.03,0.67,4


In [79]:
df_document_topic['topic'].value_counts().to_frame()

,topic
4,265
0,259
2,215
1,184
3,77


In [80]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model_lda.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[ 9.9072546   0.20099944  6.66848972 ...  5.16159755 10.43774789
  10.51465353]
 [ 0.21057734  0.20091444  1.34381103 ...  0.24317919 10.4637319
  36.05301456]
 [ 0.20228558 12.65342947  0.20083052 ...  5.71581304 37.3957062
   0.20135795]
 [ 0.20206483  4.74260247  0.2011949  ...  2.95687826  0.20134865
   0.20044116]
 [ 6.47781765  0.20205418  4.58567383 ...  3.92253197 13.50146536
  10.0305328 ]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,9.907255,0.200999,6.668490,0.206901,3.659800,0.201530,0.203382,0.202382,0.201428,9.840974,...,7.976289,7.465275,0.304745,0.205238,1.553273,15.727988,4.524160,5.161598,10.437748,10.514654
Topic1,0.210577,0.200914,1.343811,5.525863,4.071087,14.009033,2.302451,6.051221,1.310576,0.204440,...,5.075340,2.987083,0.201529,5.574412,0.203576,23.012678,0.203162,0.243179,10.463732,36.053015
Topic2,0.202286,12.653429,0.200831,10.591380,11.192775,0.201445,0.202084,0.201746,19.019406,12.791958,...,2.545625,1.609294,20.091859,1.813716,1.659679,0.202387,8.214153,5.715813,37.395706,0.201358
Topic3,0.202065,4.742602,0.201195,0.201126,4.168781,0.200688,0.201578,0.206114,0.205549,0.205148,...,0.200906,0.203493,0.201454,0.201019,0.200046,12.566943,0.201410,2.956878,0.201349,0.200441
Topic4,6.477818,0.202054,4.585674,0.474730,1.907558,2.387304,10.090505,7.338537,1.263041,4.957481,...,0.201839,1.734854,0.200412,7.205615,8.383427,82.490004,0.857115,3.922532,13.501465,10.030533


In [81]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model_lda, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,watch,great,band,look,wear,light,comfort,price,use,nice,day,buy,better,easi,time
Topic 1,watch,like,time,look,color,realli,day,love,face,year,want,read,band,good,larg
Topic 2,watch,love,look,like,wrist,fit,size,beauti,nice,invicta,strap,make,buy,awesom,cheap
Topic 3,perfect,excel,expect,product,beauti,gift,qualiti,fast,broke,work,recommend,tri,came,super,paid
Topic 4,watch,good,nice,work,time,look,batteri,use,hand,qualiti,week,price,far,month,second
